In [1]:
!pip install pytube -q
!pip install scenedetect -q
!pip install openai-whisper -q
!pip install langchain-openai -q
!pip install langchain -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 53.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 30.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 24.9 MB/s eta 0:00:00
ERROR: pip's de

In [ ]:
from pytube import YouTube
import moviepy.editor as mp

# Replace 'YOUR_VIDEO_URL' with the YouTube video URL you want to download
video_url = 'https://www.youtube.com/watch?v=7q2ulB0dhMk'
yt = YouTube(video_url)

videoPath = 'downloaded_video.mp4'
audioPath = 'downloaded_audio.wav'
# Select the desired stream and download it
video_stream = yt.streams.filter(file_extension='mp4').first()
video_stream.download(filename=videoPath)

def extract_audio(video_path, audio_path="extracted_audio.wav"):
    video_clip = mp.VideoFileClip(video_path)
    video_clip.audio.write_audiofile(audio_path)
    return audio_path

audio_path = extract_audio(video_path=videoPath, audio_path=audioPath)



MoviePy - Writing audio in downloaded_audio.wav


MoviePy - Done.


In [ ]:
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector
import cv2
import whisper
import json
from IPython.display import display, Image
from PIL import Image as PILImage
import io

def format_timestamp(seconds):
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Function to find scenes, extract key frames, and get transcribed text for each scene
def process_video(video_path, audio_path, model_size="base", threshold=80.0):
    # Scene detection
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)


    scenes = scene_manager.get_scene_list()

    print(scenes)
    # Transcription
    model = whisper.load_model(model_size)
    transcription_result = model.transcribe(audio_path)

    cap = cv2.VideoCapture(video_path)
    scene_data = []

    for i, scene in enumerate(scenes):
        # Extract frame
        frame_num = scene[0].get_frames()
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        if ret:
            # Convert frame to image format for display
            is_success, buffer = cv2.imencode(".jpg", frame)
            io_buf = io.BytesIO(buffer)
            frame_image = PILImage.open(io_buf)

            # Associate text
            start, end = scene[0].get_seconds(), scene[1].get_seconds()
            scene_text_segments = [seg for seg in transcription_result["segments"] if start <= seg["start"] < end]

            scene_text = " ".join([seg["text"] for seg in transcription_result["segments"] if start <= seg["start"] < end])
            detail_srt = "\n".join([f"[{format_timestamp(seg['start'])} - {format_timestamp(seg['end'])}] {seg['text']}" for seg in scene_text_segments])

            # Append to scene data
            scene_data.append((frame_image, scene_text, detail_srt))

    cap.release()
    return scene_data
video_path = 'downloaded_video.mp4'
audio_path = 'downloaded_audio.wav'

# Replace with your video file path
scenes_with_text = process_video(video_path, audio_path)

# Print each scene's key frame and text
# for i, (image, text) in enumerate(scenes_with_text):
#     print(f"Scene {i+1}:")
#     display(image)
#     print(f"Text: {text}\n")

ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 30.000 FPS, resolution: 640 x 360
INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 320 x 180
INFO:pyscenedetect:Detecting scenes...


[(00:00:00.000 [frame=0, fps=30.000], 00:00:02.033 [frame=61, fps=30.000]), (00:00:02.033 [frame=61, fps=30.000], 00:00:06.733 [frame=202, fps=30.000]), (00:00:06.733 [frame=202, fps=30.000], 00:05:17.733 [frame=9532, fps=30.000]), (00:05:17.733 [frame=9532, fps=30.000], 00:08:09.833 [frame=14695, fps=30.000]), (00:08:09.833 [frame=14695, fps=30.000], 00:09:23.933 [frame=16918, fps=30.000]), (00:09:23.933 [frame=16918, fps=30.000], 00:10:02.233 [frame=18067, fps=30.000]), (00:10:02.233 [frame=18067, fps=30.000], 00:10:36.100 [frame=19083, fps=30.000]), (00:10:36.100 [frame=19083, fps=30.000], 00:13:33.000 [frame=24390, fps=30.000]), (00:13:33.000 [frame=24390, fps=30.000], 00:13:43.833 [frame=24715, fps=30.000]), (00:13:43.833 [frame=24715, fps=30.000], 00:14:07.433 [frame=25423, fps=30.000]), (00:14:07.433 [frame=25423, fps=30.000], 00:14:39.167 [frame=26375, fps=30.000]), (00:14:39.167 [frame=26375, fps=30.000], 00:17:28.133 [frame=31444, fps=30.000]), (00:17:28.133 [frame=31444, fps

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



In [ ]:
!pip uninstall tensorflow-probability -y -q
!pip uninstall typing-extensions -y --quiet
!pip install typing_extensions==4.7.1 --upgrade


  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)


In [ ]:
import os

from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

def summarize_text(text):
  # Define prompt
  prompt_template = """Providing text of previous screne, current scene and next scene of video clips.Write a concise summary of the following in zh-hant:
  "{text}"
  CONCISE SUMMARY:"""
  prompt = PromptTemplate.from_template(prompt_template)

  # Define LLM chain
  chat_model = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-16k", openai_api_key=userdata.get('OPENAI_API_KEY'))

  ai_message = chat_model.invoke([prompt.format(text=text)])

  return ai_message.content

def prepare_texts_for_summarization(scene_texts):
    combined_texts = []
    for i in range(len(scene_texts)):
        prev_text = scene_texts[i - 1] if i > 0 else ""
        current_text = scene_texts[i]
        next_text = scene_texts[i + 1] if i < len(scene_texts) - 1 else ""
        combined_text = f"Previous Scene: {prev_text}\nCurrent Scene: {current_text}\nNext Scene: {next_text}"
        combined_texts.append(combined_text)
    return combined_texts



combined_texts = prepare_texts_for_summarization(scenes_with_text)
scene_summaries = [summarize_text(text) for text in combined_texts]

# for i, ((image, _), summary) in enumerate(zip(scenes_with_text, scene_summaries)):
#     print(f"Scene {i+1}:")
#     display(image)
#     print(f"Summary: {summary}\n")


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 23206 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
from IPython.core.display import display, HTML
import base64
import io

def display_scenes_in_html(scenes_with_text, scene_summaries):
    html_output = ""

    for i, ((image, _, detail_srt), summary) in enumerate(zip(scenes_with_text, scene_summaries)):
        # Convert PIL Image to Data URL
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        img_data = buffered.getvalue()
        img_data = base64.b64encode(img_data).decode()
        img_url = f"data:image/jpeg;base64,{img_data}"

        # Escape HTML characters in detail_srt
        detail_srt_html = "<br>".join(detail_srt.split("\n"))

        # Create HTML content
        html_output += f"""
        <div style='display: flex; align-items: flex-start; margin-bottom: 20px;'>
            <img src="{img_url}" style="max-width: 300px; max-height: 300px; margin-right: 20px;">
            <div style='flex: 1;'>
                <h3>Scene {i+1}:</h3>
                <p style='word-wrap: break-word;'>{summary}</p>
                <button onclick="document.getElementById('detail-srt-{i}').style.display = document.getElementById('detail-srt-{i}').style.display === 'none' ? 'block' : 'none';">Toggle Details</button>
                <div id='detail-srt-{i}' style='display: none; word-wrap: break-word; margin-top: 10px;'>
                    {detail_srt_html}
                </div>
            </div>
        </div>
        """

    # Display the final HTML
    display(HTML(html_output))

# Use this function to display scenes and summaries
display_scenes_in_html(scenes_with_text, scene_summaries)